In [21]:
import pandas as pd
import numpy as np
import urllib2
import re
import requests
import json
import time
from BeautifulSoup import BeautifulSoup
from common_func import convert_unicode

In [23]:
podcastDf = pd.read_pickle('pkl/english_podcasts.pkl')

In [18]:
itunesUrl = 'https://itunes.apple.com/search?'
parameters = {}
parameters['entity'] = 'podcast'
parameters['attribute'] = 'titleTerm'

requestString = ''
for param in parameters:
    paramString = param + '=' + parameters[param]
    if len(requestString) == 0:
        requestString = paramString
    else:
        requestString = requestString + '&' + paramString

In [45]:
date_thresh = pd.to_datetime('Dec 7 2015')
itunes_collection_id = []
mean_rating = []
num_ratings = []
release_date = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    
    #Retry opening url 4 times before failing, sleep 15 sec in between
    for x in range(0, 4):  # try 4 times
        try:
            urllib2.urlopen(requestUrl)
            str_error = None
        except urllib2.HTTPError as str_error:
            pass

        if str_error:
            time.sleep(60)  # wait for 15 seconds before trying to fetch the data again
        else:
            break
    
        
    r = requests.get(requestUrl)
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        release_date.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    
    # get release date
    if 'releaseDate' in data.keys():
        epDate = pd.to_datetime(data['releaseDate'])
        release_date.append(epDate)
        if epDate < date_thresh:
            mean_rating.append(np.nan)
            num_ratings.append(np.nan)
        else:
            scrapeUrl = scrapeStem + str(data['collectionId'])
            soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
            thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
            thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
            mean_rating.append(thisMean)
            num_ratings.append(thisTotal)
    else:
        release_date.append(np.nan)
        scrapeUrl = scrapeStem + str(data['collectionId'])
        soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
        thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
        thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
        mean_rating.append(thisMean)
        num_ratings.append(thisTotal)
    

HTTPError: HTTP Error 503: Service Unavailable

In [49]:
print len(mean_rating)
print len(num_ratings)
print len(release_date)
print len(itunes_collection_id)

5825
5825
5826
5826


In [50]:
outDf = podcastDf[:len(mean_rating)]
outDf = outDf.assign(itunes_collection_id = itunes_collection_id[:len(mean_rating)],
                            mean_rating = mean_rating,
                            num_ratings = num_ratings,
                            release_date = release_date[:len(mean_rating)])
outDf.to_pickle('pkl/itunes_podcasts_v2_pt1.pkl')

In [51]:
len(podcastDf)

49930

In [52]:
podcastDf_1 = podcastDf[len(mean_rating):]
len(podcastDf_1)

44105

In [53]:
date_thresh = pd.to_datetime('Dec 7 2015')
itunes_collection_id = []
mean_rating = []
num_ratings = []
release_date = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf_1.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    
    #Retry opening url 4 times before failing, sleep 15 sec in between
    for x in range(0, 4):  # try 4 times
        try:
            urllib2.urlopen(requestUrl)
            str_error = None
        except urllib2.HTTPError as str_error:
            pass

        if str_error:
            time.sleep(60)  # wait for 15 seconds before trying to fetch the data again
        else:
            break
    
        
    r = requests.get(requestUrl)
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        release_date.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    
    # get release date
    if 'releaseDate' in data.keys():
        epDate = pd.to_datetime(data['releaseDate'])
        release_date.append(epDate)
        if epDate < date_thresh:
            mean_rating.append(np.nan)
            num_ratings.append(np.nan)
        else:
            scrapeUrl = scrapeStem + str(data['collectionId'])
            soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
            thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
            thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
            mean_rating.append(thisMean)
            num_ratings.append(thisTotal)
    else:
        release_date.append(np.nan)
        scrapeUrl = scrapeStem + str(data['collectionId'])
        soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
        thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
        thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
        mean_rating.append(thisMean)
        num_ratings.append(thisTotal)

HTTPError: HTTP Error 503: Service Unavailable

In [54]:
len(mean_rating)

2269

In [55]:
outDf = podcastDf_1[:len(mean_rating)]
outDf = outDf.assign(itunes_collection_id = itunes_collection_id[:len(mean_rating)],
                            mean_rating = mean_rating,
                            num_ratings = num_ratings,
                            release_date = release_date[:len(mean_rating)])
outDf.to_pickle('pkl/itunes_podcasts_v2_pt2.pkl')

In [56]:
podcastDf_2 = podcastDf_1[len(mean_rating):]
len(podcastDf_2)

date_thresh = pd.to_datetime('Dec 7 2015')
itunes_collection_id = []
mean_rating = []
num_ratings = []
release_date = []
secs = 180

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf_2.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    
    #Retry opening url 4 times before failing, sleep 15 sec in between
    for x in range(0, 4):  # try 4 times
        try:
            urllib2.urlopen(requestUrl)
            str_error = None
        except urllib2.HTTPError as str_error:
            pass

        if str_error:
            time.sleep(secs)  # wait for 15 seconds before trying to fetch the data again
        else:
            break
    
        
    r = requests.get(requestUrl)
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        release_date.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    
    # get release date
    if 'releaseDate' in data.keys():
        epDate = pd.to_datetime(data['releaseDate'])
        release_date.append(epDate)
        if epDate < date_thresh:
            mean_rating.append(np.nan)
            num_ratings.append(np.nan)
        else:
            scrapeUrl = scrapeStem + str(data['collectionId'])
            soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
            thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
            thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
            mean_rating.append(thisMean)
            num_ratings.append(thisTotal)
    else:
        release_date.append(np.nan)
        scrapeUrl = scrapeStem + str(data['collectionId'])
        soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
        thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
        thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
        mean_rating.append(thisMean)
        num_ratings.append(thisTotal)

HTTPError: HTTP Error 503: Service Unavailable

In [57]:
len(mean_rating)

15419